# Introduction to Retrieval Augmented Generation with S&P 500 news

In this notebook, you will explore how to build a simple Retrieval-Augmented Generation (RAG) pipeline using financial news articles from S&P 500 companies.

We'll start by vectorizing text data, creating a vector store using FAISS, and integrating it with OpenAI's GPT models to answer questions using retrieved information.

This workflow emulates real-world systems in finance where natural language data (news, filings, analyst reports) are used to support decision-making.

# 📌 Objectives

By the end of this notebook, students will be able to:

1. **Perform Semantic Search with Metadata Filtering:**
   - Query the provided FAISS vector store to retrieve relevant financial news articles based on natural language questions.
   - Apply optional filters using metadata such as ticker or publication date to refine search results.

2. **Enrich Data with Company Metadata:**
   - Use the `yfinance` library to retrieve company-level metadata (company name, sector, industry) for tickers in the dataset.
   - Integrate this metadata to support enhanced filtering and analysis of news data.

3. **Build a Retrieval-Augmented Generation (RAG) Pipeline:**
   - Combine retrieved news snippets as context to generate answers using OpenAI’s GPT models.
   - Construct effective prompts that guide the language model to provide concise, context-aware responses.

4. **Evaluate and Analyze RAG Outputs:**
   - Review generated answers alongside the supporting news excerpts.
   - Reflect on the strengths and limitations of the simple RAG pipeline and consider potential improvements, such as adding more filters or refining retrieval strategies.

5. **Incorporate Financial Metadata into Retrieval Context:**
   - Enrich retrieved news snippets with key financial metadata including ticker, company name, sector, and industry.
   - Format prompts that combine both text excerpts and metadata to provide richer context to the language model.

6. **Generate Context-Aware Answers Using OpenAI Models:**
   - Construct and send prompts to an LLM that leverage both news content and metadata to produce concise, informed financial analysis.

7. **Compare Answers With and Without Metadata:**
   - Evaluate the impact of including financial metadata on answer quality using criteria such as clarity, detail, accuracy, and contextual relevance.
   - Summarize findings to reflect on the role of metadata in improving retrieval-augmented generation.

## Install and Import important librairies

First, we install and import the necessary libraries for:
- Text embedding generation (sentence-transformers)
- Efficient similarity search (faiss)
- Data manipulation (pandas, numpy)
- Visualization (matplotlib)

> ℹ️ FAISS uses inner product for cosine similarity by normalizing vectors.

In [ ]:
%pip install sentence-transformers
%pip install faiss-cpu

In [ ]:
%pip install yfinance

In [ ]:
from sentence_transformers import SentenceTransformer
from collections import Counter
from sklearn.metrics.pairwise import cosine_similarity
from openai import OpenAI

import faiss
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Load news data
We load a CSV file of financial news, focusing on TITLE and SUMMARY, along with metadata like TICKER and PUBLICATION_DATE.
These will be embedded into vectors and used for semantic retrieval.

In [ ]:
K = 25

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
file_path = '/content/drive/MyDrive/Colab Notebooks/MNA/Fintech/Week 3/df_news.csv'

In [ ]:
df_news = pd.read_csv(file_path)
df_news['PUBLICATION_DATE'] = pd.to_datetime(df_news['PUBLICATION_DATE']).dt.date
display(df_news)

,TICKER,TITLE,SUMMARY,PUBLICATION_DATE,PROVIDER,URL
0,MMM,2 Dow Jones Stocks with Promising Prospects an...,The Dow Jones (^DJI) is made up of 30 of the m...,2025-05-29,StockStory,https://finance.yahoo.com/news/2-dow-jones-sto...
1,MMM,3 S&P 500 Stocks Skating on Thin Ice,The S&P 500 (^GSPC) is often seen as a benchma...,2025-05-27,StockStory,https://finance.yahoo.com/news/3-p-500-stocks-...
2,MMM,3M Rises 15.8% YTD: Should You Buy the Stock N...,"MMM is making strides in the aerospace, indust...",2025-05-22,Zacks,https://finance.yahoo.com/news/3m-rises-15-8-y...
3,MMM,Q1 Earnings Roundup: 3M (NYSE:MMM) And The Res...,Quarterly earnings results are a good time to ...,2025-05-22,StockStory,https://finance.yahoo.com/news/q1-earnings-rou...
4,MMM,3 Cash-Producing Stocks with Questionable Fund...,While strong cash flow is a key indicator of s...,2025-05-19,StockStory,https://finance.yahoo.com/news/3-cash-producin...
...,...,...,...,...,...,...
4866,ZTS,2 Dividend Stocks to Buy With $500 and Hold Fo...,Zoetis is a leading animal health company with...,2025-05-23,Motley Fool,https://www.fool.com/investing/2025/05/23/2-di...
4867,ZTS,Zoetis (NYSE:ZTS) Declares US$0.50 Dividend Pe...,Zoetis (NYSE:ZTS) recently affirmed a dividend...,2025-05-22,Simply Wall St.,https://finance.yahoo.com/news/zoetis-nyse-zts...
4868,ZTS,Jim Cramer on Zoetis (ZTS): “It Does Seem to B...,We recently published a list of Jim Cramer Tal...,2025-05-21,Insider Monkey,https://finance.yahoo.com/news/jim-cramer-zoet...
4869,ZTS,Zoetis (ZTS) Upgraded to Buy: Here's Why,Zoetis (ZTS) might move higher on growing opti...,2025-05-21,Zacks,https://finance.yahoo.com/news/zoetis-zts-upgr...


In [ ]:
df_news['EMBEDDED_TEXT'] = df_news['TITLE'] + ' : ' + df_news['SUMMARY']

In [ ]:
model = SentenceTransformer('all-MiniLM-L6-v2')

## Implement FAISS vector store
We:
- Use a pre-trained sentence transformer (all-MiniLM-L6-v2) to embed documents.
- Normalize vectors to use cosine similarity.
- Create a FAISS index and implement a basic search function.

This will allow us to retrieve relevant news snippets given a natural language question.


In [ ]:
# Load model and compute embeddings
text_embeddings = model.encode(df_news['EMBEDDED_TEXT'].tolist(), convert_to_numpy=True)

# Normalize embeddings to use cosine similarity (via inner product in FAISS)
text_embeddings = text_embeddings / np.linalg.norm(text_embeddings, axis=1, keepdims=True)

# Prepare metadata
documents = df_news['EMBEDDED_TEXT'].tolist()
metadata = [
    {
        'PUBLICATION_DATE': row['PUBLICATION_DATE'],
        'TICKER': row['TICKER'],
        'PROVIDER': row['PROVIDER']
    }
    for _, row in df_news.iterrows()
]

In [ ]:
embedding_dim = text_embeddings.shape[1]
faiss_index = faiss.IndexFlatIP(embedding_dim)  # Cosine similarity via inner product
faiss_index.add(text_embeddings)

In [ ]:
class FaissVectorStore:
    def __init__(self, model, index, embeddings, documents, metadata):
        self.model = model
        self.index = index
        self.embeddings = embeddings
        self.documents = documents
        self.metadata = metadata

    def search(self, query, k=5, metadata_filter=None):
        query_embedding = self.model.encode([query])
        query_embedding = query_embedding / np.linalg.norm(query_embedding)

        if metadata_filter:
            filtered_indices = [i for i, meta in enumerate(self.metadata) if metadata_filter(meta)]
            if not filtered_indices:
                return []
            filtered_embeddings = self.embeddings[filtered_indices]
            temp_index = faiss.IndexFlatIP(filtered_embeddings.shape[1])
            temp_index.add(filtered_embeddings)
            D, I = temp_index.search(query_embedding, k)
            indices = [filtered_indices[i] for i in I[0]]
        else:
            D, I = self.index.search(query_embedding, k)
            indices = I[0]
            D = D[0]

        results = []
        for idx, sim in zip(indices, D):
            results.append((self.documents[idx], self.metadata[idx], float(sim)))
        return results

In [ ]:
# Create FAISS-based store
faiss_store = FaissVectorStore(
    model=model,
    index=faiss_index,
    embeddings=text_embeddings,
    documents=documents,
    metadata=metadata
)

### Setup OpenAI Client

👉 **Instructions**:
- Import the `OpenAI` client from the `openai` Python library.
- You will need an **OpenAI API key** to use their models programmatically:
  - Go to [https://platform.openai.com/](https://platform.openai.com/) and sign up or log in.
  - Create an API key from your [API keys dashboard](https://platform.openai.com/account/api-keys).
  - ⚠️ **Keep your API key private** and **do not** share or hardcode it in public notebooks.
- Note that **usage of the OpenAI API is not free**. You will need to:
  - Add a payment method.
  - Monitor your usage to avoid unexpected charges.
  - Optionally set usage limits from your account settings.
- You can refer to the **course’s Study Resources** for a step-by-step guide on creating an OpenAI account and retrieving your API key.

Then:
- Initialize the client with `OpenAI(api_key="YOUR_KEY_HERE")`.
- Send a test request using `.responses.create()` and the `"gpt-4o-mini"` model with a simple prompt:

  ```python
  response = client.responses.create(
      model="gpt-4o-mini",
      input="Write a one-sentence bedtime story about a unicorn."
  )
  print(response.output_text)


In [ ]:
client = OpenAI(
  api_key=""
)

In [ ]:
response = client.responses.create(
  model="gpt-4o-mini",
  input="write a haiku about ai",
  store=True,
)

print(response.output_text);

Silent circuits hum,  
Dreams of logic intertwine—  
Thoughts beyond the code.  


In [ ]:
response = client.responses.create(
  model="gpt-4o-mini",
  input="write a one-sentence bedtime story about a unicorn",
)

print(response.output_text);

As the moonlight kissed the enchanted meadow, a gentle unicorn named Luna danced with fireflies, spreading dreams of magic and wonder across the sleeping world.


## Retrieve Additional Metadata from Yahoo Finance

👉 **Instructions**:
- We will enrich our news dataset by retrieving **company-level metadata** using the `yfinance` library.
- The goal is to map each unique stock ticker (`TICKER`) in the dataset to:
  - `COMPANY_NAME`
  - `SECTOR`
  - `INDUSTRY`

> ℹ️ `yfinance` fetches live data from Yahoo Finance. If you're running this in a cloud environment or during peak hours, expect some tickers to fail or rate limits to apply.

✅ After this step, you will have a new DataFrame (e.g. `df_meta`) with the columns `TICKER`, `COMPANY_NAME`, `SECTOR`, `INDUSTRY` that maps tickers to their company names, sectors, and industries. This metadata will be useful later to add filters and analysis based on sector or industry categories.


### Add yfinance

In [ ]:
import yfinance as yf

### Get unique tickers

In [ ]:
unique_tickers = df_news['TICKER'].dropna().unique()
meta_data = []

In [ ]:
unique_tickers

array(['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ADBE', 'AMD', 'AES', 'AFL',
       'A', 'APD', 'ABNB', 'AKAM', 'ARE', 'ALGN', 'ALLE', 'LNT', 'ALL',
       'GOOGL', 'GOOG', 'MO', 'AMZN', 'AMCR', 'AEE', 'AEP', 'AXP', 'AIG',
       'AMT', 'AWK', 'AMP', 'AME', 'AMGN', 'APH', 'ADI', 'ANSS', 'AON',
       'APA', 'APO', 'AAPL', 'AMAT', 'APTV', 'ACGL', 'ADM', 'ANET', 'AJG',
       'AIZ', 'T', 'ATO', 'ADSK', 'ADP', 'AZO', 'AVB', 'AVY', 'AXON',
       'BKR', 'BALL', 'BAC', 'BAX', 'BDX', 'BBY', 'TECH', 'BIIB', 'BLK',
       'BX', 'BK', 'BA', 'BKNG', 'BSX', 'BMY', 'AVGO', 'BR', 'BRO',
       'BLDR', 'BG', 'BXP', 'CHRW', 'CDNS', 'CZR', 'CPT', 'CPB', 'COF',
       'CAH', 'KMX', 'CCL', 'CARR', 'CAT', 'CBOE', 'CBRE', 'CDW', 'COR',
       'CNC', 'CNP', 'CF', 'CRL', 'SCHW', 'CHTR', 'CVX', 'CMG', 'CHD',
       'CI', 'CINF', 'CTAS', 'CSCO', 'C', 'CFG', 'CLX', 'CME', 'CMS',
       'KO', 'CTSH', 'COIN', 'CL', 'CMCSA', 'CAG', 'COP', 'ED', 'STZ',
       'CEG', 'COO', 'CPRT', 'GLW', 'CPAY', 'CTVA', 'CSGP', 'COST',

### Map metadata

In [ ]:
for ticker in unique_tickers:
    try:
        info = yf.Ticker(ticker).info
        meta_data.append({
            'TICKER': ticker,
            'COMPANY_NAME': info.get('longName', ''),
            'SECTOR': info.get('sector', ''),
            'INDUSTRY': info.get('industry', '')
        })
    except:
        continue

### Create a new DF with metadata

In [ ]:
df_meta = pd.DataFrame(meta_data)
display(df_meta)

,TICKER,COMPANY_NAME,SECTOR,INDUSTRY
0,MMM,3M Company,Industrials,Conglomerates
1,AOS,A. O. Smith Corporation,Industrials,Specialty Industrial Machinery
2,ABT,Abbott Laboratories,Healthcare,Medical Devices
3,ABBV,AbbVie Inc.,Healthcare,Drug Manufacturers - General
4,ACN,Accenture plc,Technology,Information Technology Services
...,...,...,...,...
485,XEL,Xcel Energy Inc.,Utilities,Utilities - Regulated Electric
486,XYL,Xylem Inc.,Industrials,Specialty Industrial Machinery
487,YUM,"Yum! Brands, Inc.",Consumer Cyclical,Restaurants
488,ZBH,"Zimmer Biomet Holdings, Inc.",Healthcare,Medical Devices


## Retrieval-Augmented Generation (RAG): Retrieve Documents and Generate Answers

👉 **Instructions**:

In this part of the assignment, your task is to build a simple Retrieval-Augmented Generation (RAG) pipeline that:

- Takes a user question as input.
- Searches the FAISS vector store to find a set of relevant financial news articles based on semantic similarity.
- Uses the retrieved news articles as context to generate a clear, concise answer to the question by interacting with the OpenAI language model.
- Returns both the generated answer and the underlying news snippets used for context.

### What you need to focus on:

- Implement a retrieval mechanism to query your vector store and obtain the top relevant documents for any question.
- Construct prompts that effectively combine retrieved news content with the user’s question to guide the language model’s response.
- Use the OpenAI API to generate answers grounded in the retrieved context.
- Organize the outputs so that for each question, you have:
  - The generated answer.
  - The collection of news excerpts used to produce that answer.

### What you will be provided:

- Helper functions to display outputs in markdown format.
- Lists of example questions covering topics, companies, and industries to test your implementation.

---

Your solution can take any form or structure you find appropriate, as long as it fulfills these core objectives. This exercise will give you hands-on experience with integrating retrieval and generation for practical applications in finance.


#### Print markdown
You can use the following function to print answers from GPT4o-mini in markdown.

In [4]:
from IPython.display import Markdown, display

def print_markdown(text):
    display(Markdown(text))

#### Predefined questions

In [ ]:
questions_topic = [
"What are the major concerns expressed in financial news about inflation?",
"How is investor sentiment described in recent financial headlines?",
"What role is artificial intelligence playing in recent finance-related news stories?"
]

questions_company = [
"How is Microsoft being portrayed in news stories about artificial intelligence?",
"What financial news headlines connect Amazon with automation or logistics?"
]

questions_industry = [
"What are the main themes emerging in financial news about the semiconductor industry?",
"What trends are being reported in the retail industry?",
"What risks or challenges are discussed in recent news about the energy industry?"
]

### Function to take a user's question, search in the FAISS vector store and return the top 5 results

In [ ]:
def generate_answer(question, vector_store, model, k=5):
    # Get semantic results
    results = vector_store.search(question, k=k)
    context = "\n\n".join([doc for doc, meta, score in results])
    # create prompt
    prompt = f"Context:\n{context}\n\nQuestion: {question}\nAnswer:"
    # call open api model
    response = model.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}]
    )
    answer = response.choices[0].message.content
    return answer, context

### Initialize the vector store

In [ ]:
vector_store = FaissVectorStore(model, faiss_index, text_embeddings, documents, metadata)

### Group all questions

In [ ]:
all_questions = questions_topic + questions_company + questions_industry

### Loop to return both the generated answer and the underlying news snippets used for context.

In [ ]:
for q in all_questions:
    answer, context = generate_answer(q, vector_store, client, k=5)
    print_markdown(f"### Question:\n{q}\n\n### Answer:\n{answer}\n\n### Context:\n{context}")

### Question:
What are the major concerns expressed in financial news about inflation?

### Answer:
Major concerns expressed in financial news about inflation include:

1. **Persistent Inflation**: There is increasing worry regarding the sustained nature of inflation, as indicated by the Federal Reserve's discussions highlighting the risk that it may not be temporary.

2. **Economic Slowdown**: The potential for economic slowdown due to high inflation is a key concern, as rising prices can impact consumer spending and overall economic growth.

3. **Food Inflation**: Food inflation is specifically dampening hopes of a rate cut, suggesting that rising food prices are having a significant impact on the economy and consumers' cost of living.

4. **Tariff Impacts**: The ongoing uncertainty and fluctuations related to tariffs are adding to inflationary pressures, which complicates the economic outlook and monetary policy decisions.

5. **Policy Response**: There is unease about how the Federal Reserve may respond to these inflationary pressures, particularly whether they may need to maintain or adjust interest rates to counteract inflation, affecting financial markets.

### Context:
Bitcoin price slips as Fed minutes flag US inflation risks : The Federal Reserve’s May policy meeting revealed mounting concern over persistent US inflation and the potential for economic slowdown.

The Weekend: Food inflation dampens hopes of a rate cut as tariff twists and turns continue : Key moments from the last seven days, plus a glimpse at the week ahead

The Weekend: Food inflation dampens hopes of a rate cut as tariff twists and turns continue : Key moments from the last seven days, plus a glimpse at the week ahead

The Weekend: Food inflation dampens hopes of a rate cut as tariff twists and turns continue : Key moments from the last seven days, plus a glimpse at the week ahead

The Weekend: Food inflation dampens hopes of a rate cut as tariff twists and turns continue : Key moments from the last seven days, plus a glimpse at the week ahead

### Question:
How is investor sentiment described in recent financial headlines?

### Answer:
Investor sentiment in recent financial headlines is characterized by a mix of optimism and caution. On one hand, certain stocks are highlighted for their strong performance, positive catalysts, and favorable price targets set by analysts, suggesting attractive upside potential. However, there is also a prevalent skepticism regarding these forecasts, as analysts may face institutional pressures that can lead to overly optimistic projections. Additionally, the disparity between bullish analyst ratings and the realities of market conditions is evident, leading to warnings about stocks that may not have solid fundamentals, thus suggesting that investors should approach these recommendations with a critical eye. Overall, while there is excitement around certain stocks, the underlying messages emphasize the importance of due diligence and a wary perspective on analyst projections.

### Context:
3 of Wall Street’s Favorite Stocks Facing Headwinds : Wall Street has set ambitious price targets for the stocks in this article. While this suggests attractive upside potential, it’s important to remain skeptical because analysts face institutional pressures that can sometimes lead to overly optimistic forecasts.

3 Hyped Up  Stocks Facing Headwinds : Great things are happening to the stocks in this article. They’re all outperforming the market over the last month because of positive catalysts such as a new product line, constructive news flow, or even a loyal Reddit fanbase.

1 of Wall Street’s Favorite Stock with Impressive Fundamentals and 2 to Think Twice About : The stocks in this article have caught Wall Street’s attention in a big way, with price targets implying returns above 20%. But investors should take these forecasts with a grain of salt because analysts typically say nice things about companies so their firms can win business in other product lines like M&A advisory.

1 Unpopular Stock that Should Get More Attention and 2 to Steer Clear Of : When Wall Street turns bearish on a stock, it’s worth paying attention. These calls stand out because analysts rarely issue grim ratings on companies for fear their firms will lose out in other business lines such as M&A advisory.

3 of Wall Street’s Favorite Stocks with Questionable Fundamentals : Wall Street is overwhelmingly bullish on the stocks in this article, with price targets suggesting significant upside potential. However, it’s worth remembering that analysts rarely issue sell ratings, partly because their firms often seek other business from the same companies they cover.

### Question:
What role is artificial intelligence playing in recent finance-related news stories?

### Answer:
Artificial intelligence (AI) is playing a significant role in recent finance-related news stories by enabling companies to enhance their operations, optimize investment strategies, and improve decision-making processes. Here are several key aspects highlighted in the stories:

1. **Increased Productivity**: AI technologies are recognized for their ability to boost productivity and reduce human error. Companies like Jack Henry are integrating AI-driven lending technologies to streamline processes in the financial sector.

2. **Investment Opportunities**: Analysts and investors are closely monitoring AI investments made by technology giants like Meta Platforms. These investments are expected to unlock new revenue streams, potentially leading to significant stock appreciation as the market begins to value these AI capabilities.

3. **Billionaire Investments**: The involvement of billionaire hedge fund managers in companies like Palantir and Upstart showcases the perceived potential of AI stocks. Analysts forecast that these companies could see substantial upside due to their innovative AI applications, such as Palantir's work with government and commercial clients and Upstart's AI-driven credit risk assessment.

4. **Comparison of AI Stocks**: The financial performance of AI-focused companies, like Palantir and Snowflake, is being scrutinized. Despite market volatility, both companies have shown strong gains, indicating investor confidence in their AI capabilities.

5. **Content Production**: AI's potential to streamline content production is highlighted through companies like Netflix. The integration of AI in their content strategy suggests a transformative impact on how content is generated and delivered, which could lead to increased viewership and revenue.

In summary, AI is emerging as a pivotal force in reshaping investment strategies, enhancing operational efficiency, and influencing the financial performance of tech companies, which collectively drives interest in AI-related stocks among investors.

### Context:
Jack Henry (JKHY) Integrates AI-Driven Lending Tech With Algebrik : We recently published a list of 12 AI News Investors Should Not Miss This Week. In this article, we are going to take a look at where Jack Henry & Associates, Inc. (NASDAQ:JKHY) stands against other AI news Investors should not miss this week. Artificial Intelligence (AI) is known to increase productivity, decrease human error, […]

This "Magnificent Seven" Stock Is Set to Skyrocket If Its AI Investments Pay Off : Meta Platforms has investments in several AI applications.  The tech giant's stock is only valued on its legacy business.  Over the past two-and-a-half years, investors have heard about various artificial intelligence (AI) investments that tech companies are making.

Billionaires Are Buying 2 Artificial Intelligence (AI) Stocks That Wall Street Analysts Say Can Soar Up to 240% : Several billionaire hedge fund managers bought shares of Palantir and/or Upstart in the first quarter -- stocks where certain analysts anticipate substantial upside.  Palantir is successfully tapping demand for artificial intelligence (AI) with government and commercial customers, but the stock trades at a very expensive valuation.  Upstart is generating attractive returns for lenders by helping them quantify credit risk with artificial intelligence, and the stock trades at a very reasonable valuation.

Better Artificial Intelligence (AI) Stock: Palantir vs. Snowflake : Shares of both Palantir and Snowflake have delivered healthy gains in 2025 despite the broader stock market weakness.  Palantir stock has shot up 63% this year despite bouts of volatility.  Palantir Technologies helps commercial and government clients integrate generative AI capabilities into their operations with its Artificial Intelligence Platform (AIP), which was launched roughly two years ago.

2 Underrated Artificial Intelligence (AI) Stocks to Buy and Hold : Generative AI can simplify and speed up many tasks, including content production.  It's easy to see the potential for Netflix, whose content strategy is integral to its success.  Netflix's creations have attracted millions of viewers and won many awards.

### Question:
How is Microsoft being portrayed in news stories about artificial intelligence?

### Answer:
The provided context does not specifically mention Microsoft or its portrayal in news stories about artificial intelligence. Therefore, I cannot provide a direct analysis regarding how Microsoft is being portrayed in such articles. However, if you have additional context or specific articles about Microsoft and its AI initiatives, I would be happy to help analyze that information.

### Context:
This "Magnificent Seven" Stock Is Set to Skyrocket If Its AI Investments Pay Off : Meta Platforms has investments in several AI applications.  The tech giant's stock is only valued on its legacy business.  Over the past two-and-a-half years, investors have heard about various artificial intelligence (AI) investments that tech companies are making.

How Salesforce has 'overcorrected' by leaning into AI : D.A. Davidson head of technology research Gil Luria joins Market Domination to discuss Salesforce (CRM) earnings and the company's trajectory. Luria says Salesforce is "too focused" on artificial intelligence (AI), as the other parts of its business "rapidly" decelerate and the company loses market share to competitors. Luria has the equivalent of a Sell rating on the stock. To watch more expert insights and analysis on the latest market action, check out more Market Domination here.

Jack Henry (JKHY) Integrates AI-Driven Lending Tech With Algebrik : We recently published a list of 12 AI News Investors Should Not Miss This Week. In this article, we are going to take a look at where Jack Henry & Associates, Inc. (NASDAQ:JKHY) stands against other AI news Investors should not miss this week. Artificial Intelligence (AI) is known to increase productivity, decrease human error, […]

2 Underrated Artificial Intelligence (AI) Stocks to Buy and Hold : Generative AI can simplify and speed up many tasks, including content production.  It's easy to see the potential for Netflix, whose content strategy is integral to its success.  Netflix's creations have attracted millions of viewers and won many awards.

Meta (META) AI Reaches 1 Billion Users, Eyes Paid Features and Subscriptions : We recently published a list of 10 AI Stocks on Wall Street’s Radar. In this article, we are going to take a look at where Meta Platforms, Inc. (NASDAQ:META) stands against other AI stocks on Wall Street’s radar. Meta Platforms, Inc. (NASDAQ:META) is a global technology company. On May 28, CNBC reported that Meta Platforms, Inc. (NASDAQ:META)’s artificial […]

### Question:
What financial news headlines connect Amazon with automation or logistics?

### Answer:
The financial news headlines that connect Amazon with automation or logistics include:

1. **Amazon's AI Roadmap With AWS CEO Garman**: This article discusses how every aspect of Amazon is leveraging artificial intelligence, which often includes automation technologies that can enhance logistics and operational efficiencies.

2. **Winners And Losers Of Q1: C.H. Robinson Worldwide (NASDAQ:CHRW) Vs The Rest Of The Air Freight and Logistics Stocks**: While this headline primarily focuses on logistics, Amazon's involvement in the logistics space, particularly with its delivery and supply chain operations, can be inferred through comparisons with other logistics companies.

These headlines highlight Amazon's emphasis on AI, which is closely tied to automation and improving logistics efficiency.

### Context:
Truist Reiterates Buy on Amazon.com (AMZN) as Q2 Revenue Tracks Ahead : We recently published a list of 10 AI Stocks on Wall Street’s Radar. In this article, we are going to take a look at where Amazon.com Inc. (NASDAQ:AMZN) stands against other AI stocks on Wall Street’s radar. Amazon.com Inc. (NASDAQ:AMZN) is an American technology company offering e-commerce, cloud computing, and other services, including digital streaming […]

Amazon's AI Roadmap With AWS CEO Garman : Every aspect of Amazon is leveraging artificial intelligence, says Matt Garman, CEO of Amazon Web Services. Garman discusses Amazon's AI roadmap and reflects on his first year in the role with Ed Ludlow on "Bloomberg Technology."

Woodward's Volumes, Automation to Drive Earnings Growth, Truist Says : Woodward's (WWD) increasing volumes, pricing, automation, and products will push its aerospace margi

Top Stock Reports for Amazon.com, Johnson & Johnson & Cisco Systems : Today's Research Daily features new research reports on 16 major stocks, including Amazon.com, Inc. (AMZN), Johnson & Johnson (JNJ) and Cisco Systems, Inc. (CSCO), as well as a micro-cap NeurAxis, Inc. (NRXS).

Winners And Losers Of Q1: C.H. Robinson Worldwide (NASDAQ:CHRW) Vs The Rest Of The Air Freight and Logistics Stocks : As the craze of earnings season draws to a close, here’s a look back at some of the most exciting (and some less so) results from Q1. Today, we are looking at air freight and logistics stocks, starting with C.H. Robinson Worldwide (NASDAQ:CHRW).

### Question:
What are the main themes emerging in financial news about the semiconductor industry?

### Answer:
The main themes emerging in financial news about the semiconductor industry, particularly regarding companies like ON Semiconductor Corp. (ON), include:

1. **International Revenue Trends**: There is a strong focus on the analysis of international revenue trends for semiconductor companies. Investors are particularly interested in how these trends influence market forecasts and the potential impact on stock performance.

2. **Earnings Performance and Investor Sentiment**: Recent articles highlight that despite some companies posting soft earnings, there seems to be a lack of concern among shareholders. This indicates a level of optimism or confidence in future performance.

3. **Increased Investor Attention**: There has been a noticeable increase in interest from investors in certain semiconductor stocks, such as ON Semiconductor. This could be driven by market conditions or specific corporate developments that are attracting attention.

4. **Stock Picks from Influential Investors**: The investment strategies of prominent investors, like Billionaire Glenn Russell Dubin, are shaping discussions within the industry. Mentioning stocks with significant upside potential suggests that investors are looking for high-growth opportunities in the semiconductor sector.

5. **Market Environment Evaluation**: The current earnings season is seen as a strategic point for identifying companies that are effectively navigating the changing business environment. This implies a broader evaluation of performance among semiconductor firms and an indication of potential winners in the market.

Overall, these themes reflect a combination of market analysis, investor sentiment, and individual stock performance within the semiconductor industry.

### Context:
Investing in ON Semiconductor Corp. (ON)? Don't Miss Assessing Its International Revenue Trends : Explore ON Semiconductor Corp.'s (ON) international revenue trends and how these numbers impact Wall Street's forecasts and what's ahead for the stock.

ON Semiconductor Corporation (ON) is Attracting Investor Attention: Here is What You Should Know : Recently, Zacks.com users have been paying close attention to ON Semiconductor Corp. (ON). This makes it worthwhile to examine what the stock has in store.

Some May Be Optimistic About ON Semiconductor's (NASDAQ:ON) Earnings : Soft earnings didn't appear to concern ON Semiconductor Corporation's ( NASDAQ:ON ) shareholders over the last week...

Spotting Winners: Vishay Intertechnology (NYSE:VSH) And Analog Semiconductors Stocks In Q1 : The end of an earnings season can be a great time to discover new stocks and assess how companies are handling the current business environment. Let’s take a look at how Vishay Intertechnology (NYSE:VSH) and the rest of the analog semiconductors stocks fared in Q1.

ON Semiconductor (ON): Among Billionaire Glenn Russell Dubin’s Stock Picks with Huge Upside Potential : We recently published a list of Billionaire Glenn Russell Dubin’s 10 Stock Picks with Huge Upside Potential. In this article, we are going to take a look at where ON Semiconductor Corporation (NASDAQ:ON) stands against Billionaire Glenn Russell Dubin’s other stock picks with huge upside potential. Glenn Russell Dubin is one of the industry’s most […]

### Question:
What trends are being reported in the retail industry?

### Answer:
The retail industry is currently experiencing several concerning trends:

1. **Volatility in Consumer Demand**: There is significant uncertainty in consumer spending, leading to fluctuations in demand for retail products. This has caused retail stocks to decline by 13.7% over the past six months, which is notably worse than the S&P 500's decline of 5.5%.

2. **Adaptation to Technological Changes**: Retailers are adapting their business models in response to changing shopping behaviors driven by technology. This adaptation is essential as consumer preferences evolve, but it also contributes to the uncertainty faced by the sector.

3. **Impact of Trade Policies**: Retail executives are dealing with the ramifications of tariffs and trade policies. They have reportedly shifted their supply chains, and many price increases have already been implemented in response to these external pressures.

4. **Economic Cycle Dependency**: The performance of consumer discretionary businesses is closely linked to economic cycles. Recently, these businesses have faced headwinds, with an industry-wide drop of 12.3%, significantly worse than the S&P 500's 2.1% decline.

Overall, the combination of increased operational costs due to tariffs, changing consumer behaviors, and economic uncertainties has created a challenging environment for retailers.

### Context:
3 Consumer Stocks That Concern Us : Retailers are adapting their business models as technology changes how people shop. Still, demand can be volatile as the industry is exposed to the ups and downs of consumer spending. This has stirred some uncertainty lately as retail stocks have tumbled by 13.7% over the past six months. This performance was worse than the S&P 500’s 5.5% loss.

Retailers, Ducking Trade-War Curveballs, Stick to Their Plans : As legal rulings roll in on Trump’s tariff policies, retail executives say they have shifted their supply chains and many price increases already have hit shelves.

3 Consumer Stocks Skating on Thin Ice : The performance of consumer discretionary businesses is closely linked to economic cycles. Over the past six months, it seems like demand trends are working against their favor as the industry has tumbled by 12.3%. This drop was significantly worse than the S&P 500’s 2.1% decline.

Packaging Corporation of America (NYSE:PKG) Hasn't Managed To Accelerate Its Returns : What trends should we look for it we want to identify stocks that can multiply in value over the long term? Amongst...

Air Products and Chemicals (NYSE:APD) Will Be Hoping To Turn Its Returns On Capital Around : To find a multi-bagger stock, what are the underlying trends we should look for in a business? Firstly, we'll want to...

### Question:
What risks or challenges are discussed in recent news about the energy industry?

### Answer:
Recent news about the energy industry highlights several key risks and challenges, particularly for the renewable energy sector and oilfield services. 

1. **Legislative Changes**: The U.S. Congress is advancing a bill that could repeal important subsidies for renewable energy. This legislative change poses a significant risk to the economic viability of renewable energy projects, potentially making them uneconomical and prompting a crash in renewable energy stocks.

2. **Price Fluctuations**: The oilfield service sector, represented by companies like SLB, HAL, and BKR, faces challenges due to sliding oil prices. This decline in prices can lead to reduced profitability and budget cuts in drilling operations.

3. **Increase in Tariffs**: Rising tariffs are complicating the operational landscape for energy companies. As tariffs increase, costs related to imports and materials may rise, affecting profit margins and global competitiveness.

4. **Budget Constraints**: With reduced drilling budgets, companies in the oilfield service sector may have to navigate a more challenging financial environment, limiting their operational capacity and future investments.

5. **Market Demand Shifts**: Companies are also grappling with fluctuating demand for traditional energy sources versus renewable energy. As the market evolves, businesses must adapt or risk being left behind.

These combined factors create a highly uncertain environment for stakeholders in both the renewable energy and oilfield service sectors.

### Context:
Renewable Energy Stocks Crash as U.S. Advances Bill That Could Decimate the Industry : Congress is pushing forward a bill that could upend the renewable energy industry.  Just as companies have ramped up production and renewable electricity generation in the U.S., those projects may become uneconomical.  The news was about as bad as it could get for renewable energy stocks this week as the U.S. House of Representatives early Thursday passed a bill that will repeal some of the most important subsidies for the industry if it becomes law.

Renewable Energy Stocks Crash as U.S. Advances Bill That Could Decimate the Industry : Congress is pushing forward a bill that could upend the renewable energy industry.  Just as companies have ramped up production and renewable electricity generation in the U.S., those projects may become uneconomical.  The news was about as bad as it could get for renewable energy stocks this week as the U.S. House of Representatives early Thursday passed a bill that will repeal some of the most important subsidies for the industry if it becomes law.

Tariffs, Prices, and Pain: What's Next for Oilfield Service? : The likes of SLB, HAL and BKR face a tough future as oil prices slide, tariffs rise and drilling budgets shrink - can LNG and AI demand offer enough support?

Tariffs, Prices, and Pain: What's Next for Oilfield Service? : The likes of SLB, HAL and BKR face a tough future as oil prices slide, tariffs rise and drilling budgets shrink - can LNG and AI demand offer enough support?

3 American Companies Investors Need to Know Amid Trump's Tariff Wars : Copper is a critical metal for the U.S. industrial economy.  This American appliance maker expects the Trump administration to close loopholes that will improve its competitive positioning.  It's difficult to predict precisely what the tariff landscape will look like when the dust settles on the trade conflict, but we can say some things with a high degree of certainty.

## Analysis & Questions - Section 1

### Analysis and Reflection on Retrieval and Generation Results
After running the RAG pipeline and obtaining answers along with their supporting news excerpts, take some time to carefully review both the generated responses and the retrieved contexts.

- **For each question, read the answer and then the corresponding news snippets used as context.**

- Reflect on the following points and document your observations:
1. **Relevance**
2. **Completeness**  
3. **Bias or Noise**
4. **Consistency**  
5. **Improvement Ideas**   

and answer the questions below:

**Initial analysis**

**Q1: What are the major concerns expressed in financial news about inflation?**

In this case, the vector database returned four identical results, ranked from second to fifth in relevance. This suggests the context could have been improved, potentially by refining the query.
The response outlines five major concerns highlighted in financial news, analyzing and extracting key points from the two articles provided. Notably, item four discusses the impact of tariffs, which is not mentioned in the contextual news. To ensure the answer is strictly based on the provided context, the prompt may need adjustment.

**Q2: How is investor sentiment described in recent financial headlines?**

All articles in the context are distinct. The question seeks insights into investor sentiment. While the articles are thematically related, they lack specific references to companies, sectors, or detailed information.
As a result, the response offers a generalized market sentiment without delving into underlying causes, an important aspect for investors seeking deeper understanding.

**Q3: What role is artificial intelligence playing in recent finance-related news stories?**

The context includes information on stock purchases and recommendations for companies positioned to benefit from AI.
The response addresses multiple key topics and provides a broad perspective on AI's impact across various sectors.

**Q4: How is Microsoft being portrayed in news stories about artificial intelligence?**

The question directly seeks information about Microsoft, but the context does not contain specific details about the company. Consequently, the response is not helpful.

**Q5: What financial news headlines connect Amazon with automation or logistics?**

The question aims to identify news linking Amazon to automation or logistics. However, not all contextual articles mention Amazon, which may introduce irrelevant information.
Given only the provided context, it's difficult to assess the accuracy of the response, particularly point two, as I lack domain expertise to determine its relevance to Amazon.

**Q6: What are the main themes emerging in financial news about the semiconductor industry?**

Most articles in the context focus on ON Semiconductor. The response highlights several relevant aspects of these articles, which is valuable but limited, as the question seeks insights across the entire industry.

**Q7: What trends are being reported in the retail industry?**

The context is strong, especially the first three results, which clearly reflect trends in the retail sector. The response effectively covers several important and relevant aspects that address the question.

**Q8: What risks or challenges are discussed in recent news about the energy industry?**

Articles 1 and 2, as well as 3 and 4, are duplicates. While they are partially relevant to the topic, the duplication introduces noise and some content focuses on metals not directly tied to energy use.

To improve results, duplicate filtering and query expansion could be employed to generate multiple queries covering all facets of the question, enriching the response.
Depending on the topic and the diversity of returned articles, the number of contextual results in the prompt could be dynamic, some queries yield many relevant articles, while others result in duplication.
Additionally, sourcing context from external databases may be beneficial when the primary one lacks relevant information.


#### **Question 1.** How well do the retrieved news snippets support the generated answer? Are the key facts or themes in the answer clearly grounded in the context?

In some cases, the contextual news was relevant and useful; however, overall, I encountered duplicated results, articles that were not closely aligned with the question, and a lack of diversity to fully address the key aspects that might interest the person asking. This was partly due to my decision to limit the number of contextual articles to five and to use a basic prompt, which I acknowledge could be refined. Still, I wanted to test the system under these conditions to evaluate its performance.

The language model made a genuine effort to generate a meaningful response based on the context, though at times it introduced information whose accuracy I question and would need to verify. Again, this could likely be resolved with a more precise prompt.


#### **Question 2.** Does the answer fully address the question, or does it leave important aspects out? Consider if the retrieved context provided enough information to generate a thorough response.

In many cases, the responses left several important aspects unaddressed. The context provided was not optimal, yet the language model extracted a considerable amount of insight from what it was given. I attribute the overall quality of the responses primarily to the limitations of the context itself.
  

#### **Question 3.** Are there any irrelevant or misleading snippets retrieved that may have influenced the answer? How might this affect the quality of the output?

Undoubtedly, the poor quality of the context had a direct impact on the quality of the responses. This could have been mitigated by designing the prompt to instruct the model to analyze the context and select only the most relevant or directly related information to the question, potentially resulting in shorter but more precise answers. In some cases, I suspect that setting a limit on the number of articles used as context might have led to more focused and effective responses.




#### **Question 4.**  Do the news snippets show consistent information, or are there conflicting viewpoints? How does the LLM handle potential contradictions in the context?

In most cases, the news articles do not present a unified perspective, some even contradict one another. Additionally, many responses focus solely on a single company, despite the question requiring insights across an entire industry, resulting in inconsistencies.
To improve this using the language model, I see two potential approaches. The first involves applying query expansion: using the LLM to interpret the original question and generate sub-questions that explore relevant themes and viewpoints. Then, the LLM could be prompted to generate a comprehensive response based on grouped contextual inputs, ensuring coverage of all aspects requested from the vector database defined in the initial prompt.
The second option would be to craft a more specific prompt instructing the LLM to resolve conflicts within the context by identifying and prioritizing the most relevant argument, discarding information that does not support it. However, this approach might overlook valuable content that could enrich the analysis.

  

#### **Question 5.**  Based on your observations, suggest ways the retrieval or generation process could be improved (e.g., better filtering, adjusting `k`, refining prompt design).

I would definitely apply query expansion to better interpret the question and generate sub-questions that cover all relevant areas needed to formulate a comprehensive response.

I would also remove the fixed limit on the number of results retrieved from semantic search and implement filtering to eliminate duplicates.

Additionally, I would refine the prompt to ensure the model relies strictly on the information provided in the context, avoiding any fabrication or inference beyond what is explicitly mentioned. The model should prioritize using only the context that is strongly aligned with the question, favoring concise or even empty responses over inaccurate ones.

Finally, I would consider incorporating an alternative data source if the current database yields too few useful results.


## 🧠 Retrieval-Augmented Generation (RAG) v2: Adding Financial Metadata to Improve Generation

👉 **Instructions**:

In this part of the assignment, you’ll enhance your Retrieval-Augmented Generation (RAG) pipeline by incorporating *financial metadata* to provide more contextually rich answers.

Your goal is to evaluate whether metadata such as **company name**, **sector**, and **industry** helps the LLM generate **more accurate and grounded answers** to financial questions.

---

### ✅ What your updated pipeline should do:

- Retrieve relevant financial news articles using semantic similarity with FAISS.
- Enrich each retrieved document with financial metadata:
  - Ticker symbol
  - Full company name
  - Sector (e.g., Technology, Energy)
  - Industry (e.g., Semiconductors, Retail)
- Construct prompts that include both:
  - Retrieved news text
  - Associated metadata
- Send the prompt to the OpenAI model to generate an informed response.
- Return:
  - The final answer
  - The exact set of contextual documents used to produce that answer

---

### 🧪 Evaluation and Comparison:

You will test your improved RAG pipeline on the same three types of questions provided earlier:
- **Topic-focused** (e.g., inflation, interest rates)
- **Company-focused** (e.g., questions about Tesla, Nvidia)
- **Industry-focused** (e.g., semiconductors, utilities)


In [ ]:
def generate_enriched_answer(question, vector_store, model, df_meta, k=5):
    # Get semantic search results
    results = vector_store.search(question, k=k)
    enriched_contexts = []
    # Loop vector search results
    for doc, meta, score in results:
        # Get metadata by ticker
        ticker = meta.get('TICKER', '')
        company_info = df_meta[df_meta['TICKER'] == ticker]
        # Add metadata
        if not company_info.empty:
            company_name = company_info.iloc[0]['COMPANY_NAME']
            sector = company_info.iloc[0]['SECTOR']
            industry = company_info.iloc[0]['INDUSTRY']
        else:
            # empty metadata
            company_name = sector = industry = ''
        # create enriched context including metadata
        enriched_doc = f"Ticker: {ticker}\nCompany Name: {company_name}\nSector: {sector}\nIndustry: {industry}\nNews: {doc}"
        enriched_contexts.append(enriched_doc)
    enriched_context = "\n\n".join(enriched_contexts)
    # Create prompt including enriched context
    prompt = f"Context:\n{enriched_context}\n\nQuestion: {question}\nAnswer:"
    # call open api model
    response = model.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}]
    )
    answer = response.choices[0].message.content
    return answer, enriched_context

In [ ]:
for q in all_questions:
    enriched_answer, enriched_context = generate_enriched_answer(q, vector_store, client, df_meta, k=5)
    print_markdown(f"### Question:\n{q}\n\n### Enriched Answer:\n{enriched_answer}\n\n### Enriched Context:\n{enriched_context}")

### Question:
What are the major concerns expressed in financial news about inflation?

### Enriched Answer:
The major concerns expressed in financial news about inflation include mounting worries over persistent US inflation and its potential impact on the economy, as highlighted by the Federal Reserve's May policy meeting. Additionally, food inflation is dampening hopes for a rate cut, suggesting that rising food prices may complicate monetary policy decisions. The overall sentiment indicates apprehension regarding the economic slowdown that could result from ongoing inflationary pressures, affecting various sectors such as financial services, technology, and consumer cyclical industries.

### Enriched Context:
Ticker: BLK
Company Name: BlackRock, Inc.
Sector: Financial Services
Industry: Asset Management
News: Bitcoin price slips as Fed minutes flag US inflation risks : The Federal Reserve’s May policy meeting revealed mounting concern over persistent US inflation and the potential for economic slowdown.

Ticker: TSLA
Company Name: Tesla, Inc.
Sector: Consumer Cyclical
Industry: Auto Manufacturers
News: The Weekend: Food inflation dampens hopes of a rate cut as tariff twists and turns continue : Key moments from the last seven days, plus a glimpse at the week ahead

Ticker: NVDA
Company Name: NVIDIA Corporation
Sector: Technology
Industry: Semiconductors
News: The Weekend: Food inflation dampens hopes of a rate cut as tariff twists and turns continue : Key moments from the last seven days, plus a glimpse at the week ahead

Ticker: LULU
Company Name: lululemon athletica inc.
Sector: Consumer Cyclical
Industry: Apparel Retail
News: The Weekend: Food inflation dampens hopes of a rate cut as tariff twists and turns continue : Key moments from the last seven days, plus a glimpse at the week ahead

Ticker: AVGO
Company Name: Broadcom Inc.
Sector: Technology
Industry: Semiconductors
News: The Weekend: Food inflation dampens hopes of a rate cut as tariff twists and turns continue : Key moments from the last seven days, plus a glimpse at the week ahead

### Question:
How is investor sentiment described in recent financial headlines?

### Enriched Answer:
Investor sentiment in recent financial headlines is characterized by a mix of optimism and skepticism. While many stocks are highlighted with ambitious price targets and perceived attractive upside potential, there is a concurrent warning about the need for caution. Analysts may have biases due to institutional pressures or the desire for business from the companies they cover, leading to potentially overly optimistic forecasts. Additionally, bearish sentiments on certain stocks are noted as significant, especially since analysts generally avoid issuing negative ratings out of concern for losing future business opportunities. Overall, there is a prevailing theme of being aware of the limitations of analyst forecasts and the underlying fundamentals of the companies being discussed.

### Enriched Context:
Ticker: KMX
Company Name: CarMax, Inc.
Sector: Consumer Cyclical
Industry: Auto & Truck Dealerships
News: 3 of Wall Street’s Favorite Stocks Facing Headwinds : Wall Street has set ambitious price targets for the stocks in this article. While this suggests attractive upside potential, it’s important to remain skeptical because analysts face institutional pressures that can sometimes lead to overly optimistic forecasts.

Ticker: MCHP
Company Name: Microchip Technology Incorporated
Sector: Technology
Industry: Semiconductors
News: 3 Hyped Up  Stocks Facing Headwinds : Great things are happening to the stocks in this article. They’re all outperforming the market over the last month because of positive catalysts such as a new product line, constructive news flow, or even a loyal Reddit fanbase.

Ticker: MPWR
Company Name: Monolithic Power Systems, Inc.
Sector: Technology
Industry: Semiconductors
News: 1 of Wall Street’s Favorite Stock with Impressive Fundamentals and 2 to Think Twice About : The stocks in this article have caught Wall Street’s attention in a big way, with price targets implying returns above 20%. But investors should take these forecasts with a grain of salt because analysts typically say nice things about companies so their firms can win business in other product lines like M&A advisory.

Ticker: DRI
Company Name: Darden Restaurants, Inc.
Sector: Consumer Cyclical
Industry: Restaurants
News: 1 Unpopular Stock that Should Get More Attention and 2 to Steer Clear Of : When Wall Street turns bearish on a stock, it’s worth paying attention. These calls stand out because analysts rarely issue grim ratings on companies for fear their firms will lose out in other business lines such as M&A advisory.

Ticker: RVTY
Company Name: Revvity, Inc.
Sector: Healthcare
Industry: Diagnostics & Research
News: 3 of Wall Street’s Favorite Stocks with Questionable Fundamentals : Wall Street is overwhelmingly bullish on the stocks in this article, with price targets suggesting significant upside potential. However, it’s worth remembering that analysts rarely issue sell ratings, partly because their firms often seek other business from the same companies they cover.

### Question:
What role is artificial intelligence playing in recent finance-related news stories?

### Enriched Answer:
Artificial intelligence (AI) is emerging as a critical driver of innovation and investment across various sectors, significantly shaping recent finance-related news stories. Here are some key roles AI is playing:

1. **Enhancing Productivity and Reducing Errors**: Companies like Jack Henry & Associates, Inc. (JKHY) are integrating AI-driven technologies to enhance productivity and minimize human error, particularly in lending and financial services. This integration reflects a broader trend of leveraging AI to streamline operations and improve service quality in the finance sector.

2. **Investment Strategies**: Major firms are making substantial investments in AI technologies, with companies like Meta Platforms, Inc. (META) focusing on AI applications to boost their future growth prospects. Investors are closely watching these developments, as the potential of AI to enhance traditional business models may drive stock prices significantly higher.

3. **Attracting Institutional Investments**: Stocks like Palantir Technologies Inc. (PLTR) are gaining attention from billionaire hedge fund managers and analysts who foresee considerable upside potential due to the demand for AI solutions. This interest suggests that AI capabilities are not just enhancing current offerings but are also seen as critical components for future success in the marketplace.

4. **Market Performance**: Palantir has experienced substantial stock price increases, attributed to its ability to integrate AI capabilities into its offerings. The growth reflects investor confidence in companies that successfully implement AI, even amidst broader market volatility.

5. **Content Production and Engagement**: In the entertainment sector, companies like Netflix, Inc. (NFLX) are utilizing generative AI to simplify content production processes. As content creation becomes increasingly complex and competitive, AI is viewed as a tool that can improve efficiency and drive higher engagement with audiences.

Overall, AI is playing a transformative role in how companies operate, attract investments, and engage with consumers, positioning itself as a critical element in both financial performance and strategic planning across various industries.

### Enriched Context:
Ticker: JKHY
Company Name: Jack Henry & Associates, Inc.
Sector: Technology
Industry: Information Technology Services
News: Jack Henry (JKHY) Integrates AI-Driven Lending Tech With Algebrik : We recently published a list of 12 AI News Investors Should Not Miss This Week. In this article, we are going to take a look at where Jack Henry & Associates, Inc. (NASDAQ:JKHY) stands against other AI news Investors should not miss this week. Artificial Intelligence (AI) is known to increase productivity, decrease human error, […]

Ticker: META
Company Name: Meta Platforms, Inc.
Sector: Communication Services
Industry: Internet Content & Information
News: This "Magnificent Seven" Stock Is Set to Skyrocket If Its AI Investments Pay Off : Meta Platforms has investments in several AI applications.  The tech giant's stock is only valued on its legacy business.  Over the past two-and-a-half years, investors have heard about various artificial intelligence (AI) investments that tech companies are making.

Ticker: PLTR
Company Name: Palantir Technologies Inc.
Sector: Technology
Industry: Software - Infrastructure
News: Billionaires Are Buying 2 Artificial Intelligence (AI) Stocks That Wall Street Analysts Say Can Soar Up to 240% : Several billionaire hedge fund managers bought shares of Palantir and/or Upstart in the first quarter -- stocks where certain analysts anticipate substantial upside.  Palantir is successfully tapping demand for artificial intelligence (AI) with government and commercial customers, but the stock trades at a very expensive valuation.  Upstart is generating attractive returns for lenders by helping them quantify credit risk with artificial intelligence, and the stock trades at a very reasonable valuation.

Ticker: PLTR
Company Name: Palantir Technologies Inc.
Sector: Technology
Industry: Software - Infrastructure
News: Better Artificial Intelligence (AI) Stock: Palantir vs. Snowflake : Shares of both Palantir and Snowflake have delivered healthy gains in 2025 despite the broader stock market weakness.  Palantir stock has shot up 63% this year despite bouts of volatility.  Palantir Technologies helps commercial and government clients integrate generative AI capabilities into their operations with its Artificial Intelligence Platform (AIP), which was launched roughly two years ago.

Ticker: NFLX
Company Name: Netflix, Inc.
Sector: Communication Services
Industry: Entertainment
News: 2 Underrated Artificial Intelligence (AI) Stocks to Buy and Hold : Generative AI can simplify and speed up many tasks, including content production.  It's easy to see the potential for Netflix, whose content strategy is integral to its success.  Netflix's creations have attracted millions of viewers and won many awards.

### Question:
How is Microsoft being portrayed in news stories about artificial intelligence?

### Enriched Answer:
The provided context does not include any news stories or references about Microsoft and its portrayal in relation to artificial intelligence. Therefore, I cannot provide an answer regarding how Microsoft is being portrayed in those discussions. If you have specific news articles or sources related to Microsoft and AI, feel free to share, and I'll be glad to help analyze them.

### Enriched Context:
Ticker: META
Company Name: Meta Platforms, Inc.
Sector: Communication Services
Industry: Internet Content & Information
News: This "Magnificent Seven" Stock Is Set to Skyrocket If Its AI Investments Pay Off : Meta Platforms has investments in several AI applications.  The tech giant's stock is only valued on its legacy business.  Over the past two-and-a-half years, investors have heard about various artificial intelligence (AI) investments that tech companies are making.

Ticker: CRM
Company Name: Salesforce, Inc.
Sector: Technology
Industry: Software - Application
News: How Salesforce has 'overcorrected' by leaning into AI : D.A. Davidson head of technology research Gil Luria joins Market Domination to discuss Salesforce (CRM) earnings and the company's trajectory. Luria says Salesforce is "too focused" on artificial intelligence (AI), as the other parts of its business "rapidly" decelerate and the company loses market share to competitors. Luria has the equivalent of a Sell rating on the stock. To watch more expert insights and analysis on the latest market action, check out more Market Domination here.

Ticker: JKHY
Company Name: Jack Henry & Associates, Inc.
Sector: Technology
Industry: Information Technology Services
News: Jack Henry (JKHY) Integrates AI-Driven Lending Tech With Algebrik : We recently published a list of 12 AI News Investors Should Not Miss This Week. In this article, we are going to take a look at where Jack Henry & Associates, Inc. (NASDAQ:JKHY) stands against other AI news Investors should not miss this week. Artificial Intelligence (AI) is known to increase productivity, decrease human error, […]

Ticker: NFLX
Company Name: Netflix, Inc.
Sector: Communication Services
Industry: Entertainment
News: 2 Underrated Artificial Intelligence (AI) Stocks to Buy and Hold : Generative AI can simplify and speed up many tasks, including content production.  It's easy to see the potential for Netflix, whose content strategy is integral to its success.  Netflix's creations have attracted millions of viewers and won many awards.

Ticker: META
Company Name: Meta Platforms, Inc.
Sector: Communication Services
Industry: Internet Content & Information
News: Meta (META) AI Reaches 1 Billion Users, Eyes Paid Features and Subscriptions : We recently published a list of 10 AI Stocks on Wall Street’s Radar. In this article, we are going to take a look at where Meta Platforms, Inc. (NASDAQ:META) stands against other AI stocks on Wall Street’s radar. Meta Platforms, Inc. (NASDAQ:META) is a global technology company. On May 28, CNBC reported that Meta Platforms, Inc. (NASDAQ:META)’s artificial […]

### Question:
What financial news headlines connect Amazon with automation or logistics?

### Enriched Answer:
The financial news headlines that connect Amazon with automation or logistics include:

1. **Amazon's AI Roadmap With AWS CEO Garman**: This article discusses how Amazon is leveraging artificial intelligence across all its operations, which includes aspects of logistics and automation within its extensive e-commerce and cloud services.

2. **Winners And Losers Of Q1: C.H. Robinson Worldwide (NASDAQ:CHRW) Vs The Rest Of The Air Freight and Logistics Stocks**: While this headline focuses on C.H. Robinson, it fits into the logistics theme as it gives insight into the performance of logistics companies, which can be influenced by Amazon's operations and the broader impact of automation in the logistics sector.

These headlines highlight Amazon's commitment to integrating automation and AI into its logistics and operational frameworks.

### Enriched Context:
Ticker: TFC
Company Name: Truist Financial Corporation
Sector: Financial Services
Industry: Banks - Regional
News: Truist Reiterates Buy on Amazon.com (AMZN) as Q2 Revenue Tracks Ahead : We recently published a list of 10 AI Stocks on Wall Street’s Radar. In this article, we are going to take a look at where Amazon.com Inc. (NASDAQ:AMZN) stands against other AI stocks on Wall Street’s radar. Amazon.com Inc. (NASDAQ:AMZN) is an American technology company offering e-commerce, cloud computing, and other services, including digital streaming […]

Ticker: AMZN
Company Name: Amazon.com, Inc.
Sector: Consumer Cyclical
Industry: Internet Retail
News: Amazon's AI Roadmap With AWS CEO Garman : Every aspect of Amazon is leveraging artificial intelligence, says Matt Garman, CEO of Amazon Web Services. Garman discusses Amazon's AI roadmap and reflects on his first year in the role with Ed Ludlow on "Bloomberg Technology."

Ticker: TFC
Company Name: Truist Financial Corporation
Sector: Financial Services
Industry: Banks - Regional
News: Woodward's Volumes, Automation to Drive Earnings Growth, Truist Says : Woodward's (WWD) increasing volumes, pricing, automation, and products will push its aerospace margi

Ticker: AON
Company Name: Aon plc
Sector: Financial Services
Industry: Insurance Brokers
News: Top Stock Reports for Amazon.com, Johnson & Johnson & Cisco Systems : Today's Research Daily features new research reports on 16 major stocks, including Amazon.com, Inc. (AMZN), Johnson & Johnson (JNJ) and Cisco Systems, Inc. (CSCO), as well as a micro-cap NeurAxis, Inc. (NRXS).

Ticker: CHRW
Company Name: C.H. Robinson Worldwide, Inc.
Sector: Industrials
Industry: Integrated Freight & Logistics
News: Winners And Losers Of Q1: C.H. Robinson Worldwide (NASDAQ:CHRW) Vs The Rest Of The Air Freight and Logistics Stocks : As the craze of earnings season draws to a close, here’s a look back at some of the most exciting (and some less so) results from Q1. Today, we are looking at air freight and logistics stocks, starting with C.H. Robinson Worldwide (NASDAQ:CHRW).

### Question:
What are the main themes emerging in financial news about the semiconductor industry?

### Enriched Answer:
The main themes emerging in financial news about the semiconductor industry, particularly regarding ON Semiconductor Corporation, include:

1. **International Revenue Trends**: There is a strong focus on how international revenue figures impact overall forecasts for companies like ON Semiconductor. Investors are keen to understand global market dynamics and how they influence profitability and growth prospects.

2. **Investor Attention and Market Sentiment**: ON Semiconductor is attracting significant investor interest, indicating a favorable market sentiment. The increased attention from investors may relate to potential growth opportunities and the company’s standing in the semiconductor space.

3. **Earnings Performance**: Reports hint at concerns surrounding soft earnings, yet there appears to be resilience among shareholders. This suggests that investors may be looking beyond short-term earnings challenges, focusing instead on long-term potential and future performance.

4. **Analyst and Expert Evaluations**: The evaluation of ON Semiconductor against broader market trends and alongside the stock picks of well-known investors (like Glenn Russell Dubin) reveals a competitive landscape where firms are assessed for upside potential in uncertain markets.

5. **Comparative Analysis Within the Sector**: There is a comparative analysis with other semiconductor companies, highlighting relative performance and opportunities amid changing market environments. This includes looking at how ON Semiconductor stacks up against peers like Analog Devices and Vishay Intertechnology in terms of growth and revenue handling.

Overall, the semiconductor sector is characterized by discussions around revenue growth strategies, market positioning, earnings outlooks, and investor sentiment shaped by broader economic conditions.

### Enriched Context:
Ticker: ON
Company Name: ON Semiconductor Corporation
Sector: Technology
Industry: Semiconductors
News: Investing in ON Semiconductor Corp. (ON)? Don't Miss Assessing Its International Revenue Trends : Explore ON Semiconductor Corp.'s (ON) international revenue trends and how these numbers impact Wall Street's forecasts and what's ahead for the stock.

Ticker: ON
Company Name: ON Semiconductor Corporation
Sector: Technology
Industry: Semiconductors
News: ON Semiconductor Corporation (ON) is Attracting Investor Attention: Here is What You Should Know : Recently, Zacks.com users have been paying close attention to ON Semiconductor Corp. (ON). This makes it worthwhile to examine what the stock has in store.

Ticker: ON
Company Name: ON Semiconductor Corporation
Sector: Technology
Industry: Semiconductors
News: Some May Be Optimistic About ON Semiconductor's (NASDAQ:ON) Earnings : Soft earnings didn't appear to concern ON Semiconductor Corporation's ( NASDAQ:ON ) shareholders over the last week...

Ticker: ADI
Company Name: Analog Devices, Inc.
Sector: Technology
Industry: Semiconductors
News: Spotting Winners: Vishay Intertechnology (NYSE:VSH) And Analog Semiconductors Stocks In Q1 : The end of an earnings season can be a great time to discover new stocks and assess how companies are handling the current business environment. Let’s take a look at how Vishay Intertechnology (NYSE:VSH) and the rest of the analog semiconductors stocks fared in Q1.

Ticker: ON
Company Name: ON Semiconductor Corporation
Sector: Technology
Industry: Semiconductors
News: ON Semiconductor (ON): Among Billionaire Glenn Russell Dubin’s Stock Picks with Huge Upside Potential : We recently published a list of Billionaire Glenn Russell Dubin’s 10 Stock Picks with Huge Upside Potential. In this article, we are going to take a look at where ON Semiconductor Corporation (NASDAQ:ON) stands against Billionaire Glenn Russell Dubin’s other stock picks with huge upside potential. Glenn Russell Dubin is one of the industry’s most […]

### Question:
What trends are being reported in the retail industry?

### Enriched Answer:
The retail industry is currently facing several concerning trends. First, there is a notable adaptation of business models by retailers as technology changes consumer shopping behaviors. This shift indicates an ongoing transformation in how products are marketed and sold.

Second, despite these adaptations, demand within the industry remains volatile, making retailers vulnerable to fluctuations in consumer spending. This uncertainty has been reflected in the overall performance of retail stocks, which have dropped by 13.7% over the past six months, significantly underperforming compared to the S&P 500’s decline of 5.5%.

Additionally, retailers are grappling with external factors such as trade tariffs, prompting many to alter their supply chains and implement price increases, which may affect consumer purchasing habits further. Overall, the trends highlight a challenging environment for retailers as they navigate economic cycles and consumer behavior changes.

### Enriched Context:
Ticker: KMX
Company Name: CarMax, Inc.
Sector: Consumer Cyclical
Industry: Auto & Truck Dealerships
News: 3 Consumer Stocks That Concern Us : Retailers are adapting their business models as technology changes how people shop. Still, demand can be volatile as the industry is exposed to the ups and downs of consumer spending. This has stirred some uncertainty lately as retail stocks have tumbled by 13.7% over the past six months. This performance was worse than the S&P 500’s 5.5% loss.

Ticker: BBY
Company Name: Best Buy Co., Inc.
Sector: Consumer Cyclical
Industry: Specialty Retail
News: Retailers, Ducking Trade-War Curveballs, Stick to Their Plans : As legal rulings roll in on Trump’s tariff policies, retail executives say they have shifted their supply chains and many price increases already have hit shelves.

Ticker: HLT
Company Name: Hilton Worldwide Holdings Inc.
Sector: Consumer Cyclical
Industry: Lodging
News: 3 Consumer Stocks Skating on Thin Ice : The performance of consumer discretionary businesses is closely linked to economic cycles. Over the past six months, it seems like demand trends are working against their favor as the industry has tumbled by 12.3%. This drop was significantly worse than the S&P 500’s 2.1% decline.

Ticker: PKG
Company Name: Packaging Corporation of America
Sector: Consumer Cyclical
Industry: Packaging & Containers
News: Packaging Corporation of America (NYSE:PKG) Hasn't Managed To Accelerate Its Returns : What trends should we look for it we want to identify stocks that can multiply in value over the long term? Amongst...

Ticker: APD
Company Name: Air Products and Chemicals, Inc.
Sector: Basic Materials
Industry: Specialty Chemicals
News: Air Products and Chemicals (NYSE:APD) Will Be Hoping To Turn Its Returns On Capital Around : To find a multi-bagger stock, what are the underlying trends we should look for in a business? Firstly, we'll want to...

### Question:
What risks or challenges are discussed in recent news about the energy industry?

### Enriched Answer:
Recent news highlights several risks and challenges facing the energy industry, particularly regarding renewable energy and oilfield services:

1. **Legislative Threats to Renewable Energy Subsidies**: The advancement of a bill by the U.S. House of Representatives aims to repeal significant subsidies for the renewable energy sector. If enacted, this legislation could render many renewable projects uneconomical, jeopardizing the financial viability of companies like NextEra Energy, Inc. (NEE) and Enphase Energy, Inc. (ENPH).

2. **Declining Oil Prices and Budget Cuts**: The oilfield services sector, including companies like Halliburton Company (HAL) and Baker Hughes Company (BKR), faces a difficult environment due to sliding oil prices. This decline is compounded by increasing tariffs and shrinking drilling budgets, raising concerns about future profitability and operational sustainability.

3. **Uncertainty from Geopolitical and Trade Conflicts**: The ongoing trade conflict, particularly related to tariffs, adds uncertainty to the market dynamics for companies reliant on critical materials like copper. For instance, Freeport-McMoRan Inc. (FCX) navigates the complexities of tariff implications on the industrial economy, which could affect competitive positioning and operational costs.

Overall, the energy industry is grappling with potential regulatory changes, volatile market conditions, and the impact of global trade policies.

### Enriched Context:
Ticker: NEE
Company Name: NextEra Energy, Inc.
Sector: Utilities
Industry: Utilities - Regulated Electric
News: Renewable Energy Stocks Crash as U.S. Advances Bill That Could Decimate the Industry : Congress is pushing forward a bill that could upend the renewable energy industry.  Just as companies have ramped up production and renewable electricity generation in the U.S., those projects may become uneconomical.  The news was about as bad as it could get for renewable energy stocks this week as the U.S. House of Representatives early Thursday passed a bill that will repeal some of the most important subsidies for the industry if it becomes law.

Ticker: ENPH
Company Name: Enphase Energy, Inc.
Sector: Technology
Industry: Solar
News: Renewable Energy Stocks Crash as U.S. Advances Bill That Could Decimate the Industry : Congress is pushing forward a bill that could upend the renewable energy industry.  Just as companies have ramped up production and renewable electricity generation in the U.S., those projects may become uneconomical.  The news was about as bad as it could get for renewable energy stocks this week as the U.S. House of Representatives early Thursday passed a bill that will repeal some of the most important subsidies for the industry if it becomes law.

Ticker: HAL
Company Name: Halliburton Company
Sector: Energy
Industry: Oil & Gas Equipment & Services
News: Tariffs, Prices, and Pain: What's Next for Oilfield Service? : The likes of SLB, HAL and BKR face a tough future as oil prices slide, tariffs rise and drilling budgets shrink - can LNG and AI demand offer enough support?

Ticker: BKR
Company Name: Baker Hughes Company
Sector: Energy
Industry: Oil & Gas Equipment & Services
News: Tariffs, Prices, and Pain: What's Next for Oilfield Service? : The likes of SLB, HAL and BKR face a tough future as oil prices slide, tariffs rise and drilling budgets shrink - can LNG and AI demand offer enough support?

Ticker: FCX
Company Name: Freeport-McMoRan Inc.
Sector: Basic Materials
Industry: Copper
News: 3 American Companies Investors Need to Know Amid Trump's Tariff Wars : Copper is a critical metal for the U.S. industrial economy.  This American appliance maker expects the Trump administration to close loopholes that will improve its competitive positioning.  It's difficult to predict precisely what the tariff landscape will look like when the dust settles on the trade conflict, but we can say some things with a high degree of certainty.

## Analysis & Questions - Section 2

### Instructions: Evaluate Answers With and Without Metadata

For each question, compare the two answers provided:
- One generated **without** metadata
- One generated **with** metadata

---

### Steps:

1. Use the following evaluation criteria:
   - Clarity
   - Detail & Depth
   - Use of Context
   - Accuracy & Grounding
   - Relevance
   - Narrrative Flow

2. For each criterion, write brief notes comparing how the answer **without metadata** performs versus the answer **with metadata**.

3. Summarize your evaluation in a markdown table with the following columns:

| Criteria       | WITHOUT METADATA            | WITH METADATA             |
|----------------|----------------------------|--------------------------|
| Clarity        | [Your brief note here]     | [Your brief note here]   |
| Detail & Depth         | [Your brief note here]     | [Your brief note here]   |
| Use of Context        | [Your brief note here]     | [Your brief note here]   |
| Accuracy & Grounding       | [Your brief note here]     | [Your brief note here]   |
| Relevance      | [Your brief note here]     | [Your brief note here]   |
| Narrative Flow      | [Your brief note here]     | [Your brief note here]   |

---

**Note:** Keep comments short and clear for easy comparison.



In [1]:
def dataframe_to_markdown(df):
    header = '| ' + ' | '.join(df.columns) + ' |'
    separator = '| ' + ' | '.join(['---'] * len(df.columns)) + ' |'
    rows = ['| ' + ' | '.join(map(str, row)) + ' |' for row in df.values]
    table_mk = '\n'.join([header, separator] + rows)
    return table_mk


In [2]:
import pandas as pd

### Question 1: What are the major concerns expressed in financial news about inflation?

**Metadata answer:**
The major concerns expressed in financial news about inflation include mounting worries over persistent US inflation and its potential impact on the economy, as highlighted by the Federal Reserve's May policy meeting. Additionally, food inflation is dampening hopes for a rate cut, suggesting that rising food prices may complicate monetary policy decisions. The overall sentiment indicates apprehension regarding the economic slowdown that could result from ongoing inflationary pressures, affecting various sectors such as financial services, technology, and consumer cyclical industries.

**Non-metadata answer:**
Major concerns expressed in financial news about inflation include:

Persistent Inflation: There is increasing worry regarding the sustained nature of inflation, as indicated by the Federal Reserve's discussions highlighting the risk that it may not be temporary.

* Economic Slowdown: The potential for economic slowdown due to high inflation is a key concern, as rising prices can impact consumer spending and overall economic growth.

* Food Inflation: Food inflation is specifically dampening hopes of a rate cut, suggesting that rising food prices are having a significant impact on the economy and consumers' cost of living.

* Tariff Impacts: The ongoing uncertainty and fluctuations related to tariffs are adding to inflationary pressures, which complicates the economic outlook and monetary policy decisions.

* Policy Response: There is unease about how the Federal Reserve may respond to these inflationary pressures, particularly whether they may need to maintain or adjust interest rates to counteract inflation, affecting financial markets.

**comparison between answers**

The response that uses metadata offers a more fluid narrative, effectively consolidating the contextual information and presenting it in a concise and highly useful manner.

In contrast, the response without metadata separates the main topics to address the question, which can also be helpful given the nature of the inquiry. However, the context does not mention fees, which raises doubts and makes the response feel less trustworthy.


In [ ]:
q_one_df = pd.DataFrame({
    'Criteria': ['Clarity', 'Detail & Depth', 'Use of Context', 'Accuracy & Grounding', 'Relevance', 'Narrative flow'],
    'Without Metadata': ["Key topics are clearly delineated", "The content is highly detailed",
                         "Generates substantial information despite limited context","The provided context does not reference tariffs",
                         "The information presented is highly pertinent" , "The flow is somewhat challenging to follow; the content could benefit from greater synthesis",],
    'With Metadata': ['Precise and to the point', 'The content is brief yet richly detailed.',
                      'Context is leveraged effectively', "The response directly and clearly addresses the questio",
                      "Highly relevant in relation to the question posed", "The narrative flows seamlessly"]
})

display(Markdown(dataframe_to_markdown(q_one_df)))

| Criteria | Without Metadata | With Metadata |
| --- | --- | --- |
| Clarity | Key topics are clearly delineated | Precise and to the point |
| Detail & Depth | The content is highly detailed | The content is brief yet richly detailed. |
| Use of Context | Generates substantial information despite limited context | Context is leveraged effectively |
| Accuracy & Grounding | The provided context does not reference tariffs | The response directly and clearly addresses the questio |
| Relevance | The information presented is highly pertinent | Highly relevant in relation to the question posed |
| Narrative flow | The flow is somewhat challenging to follow; the content could benefit from greater synthesis | The narrative flows seamlessly |

### Q2: How is investor sentiment described in recent financial headlines?

**Metadata answer:**
Investor sentiment in recent financial headlines is characterized by a mix of optimism and skepticism. While many stocks are highlighted with ambitious price targets and perceived attractive upside potential, there is a concurrent warning about the need for caution. Analysts may have biases due to institutional pressures or the desire for business from the companies they cover, leading to potentially overly optimistic forecasts. Additionally, bearish sentiments on certain stocks are noted as significant, especially since analysts generally avoid issuing negative ratings out of concern for losing future business opportunities. Overall, there is a prevailing theme of being aware of the limitations of analyst forecasts and the underlying fundamentals of the companies being discussed.

**Non-metadata answer:**
Investor sentiment in recent financial headlines is characterized by a mix of optimism and caution. On one hand, certain stocks are highlighted for their strong performance, positive catalysts, and favorable price targets set by analysts, suggesting attractive upside potential. However, there is also a prevalent skepticism regarding these forecasts, as analysts may face institutional pressures that can lead to overly optimistic projections. Additionally, the disparity between bullish analyst ratings and the realities of market conditions is evident, leading to warnings about stocks that may not have solid fundamentals, thus suggesting that investors should approach these recommendations with a critical eye. Overall, while there is excitement around certain stocks, the underlying messages emphasize the importance of due diligence and a wary perspective on analyst projections.


**comparison between answers**

The metadata answer is more robust, transparent, and actionable. It offers clearer attribution of insights to specific companies and articles

The non-metadata answer captures the general tone of headlines but is more generalized and abstract and lacks traceability,

In [9]:
q_two_df = pd.DataFrame({
    'Criteria': ['Clarity', 'Detail & Depth', 'Use of Context', 'Accuracy & Grounding', 'Relevance', 'Narrative flow'],
    'Without Metadata': ["Generally clear, but a bit more abstract ", "Covers key themes ",
                         "Uses general context from headlines","Accurate overall, but harder to verify" ,"Relevant, though broader",
                         "Flows well, but feels more like a summary than a structured argument",],
    'With Metadata': ['Clear and well-structured', 'References analyst behavior and company fundamentals',
                      'Effectively integrates contextual metadata', "Strong grounding in source material",
                      "Highly relevant to the question", "Smooth and cohesive"]
})

display(Markdown(dataframe_to_markdown(q_two_df)))

| Criteria | Without Metadata | With Metadata |
| --- | --- | --- |
| Clarity | Generally clear, but a bit more abstract  | Clear and well-structured |
| Detail & Depth | Covers key themes  | References analyst behavior and company fundamentals |
| Use of Context | Uses general context from headlines | Effectively integrates contextual metadata |
| Accuracy & Grounding | Accurate overall, but harder to verify | Strong grounding in source material |
| Relevance | Relevant, though broader | Highly relevant to the question |
| Narrative flow | Flows well, but feels more like a summary than a structured argument | Smooth and cohesive |

### Q3: What role is artificial intelligence playing in recent finance-related news stories?

**Metadata answer:**
Artificial intelligence (AI) is emerging as a critical driver of innovation and investment across various sectors, significantly shaping recent finance-related news stories. Here are some key roles AI is playing:

* Enhancing Productivity and Reducing Errors: Companies like Jack Henry & Associates, Inc. (JKHY) are integrating AI-driven technologies to enhance productivity and minimize human error, particularly in lending and financial services. This integration reflects a broader trend of leveraging AI to streamline operations and improve service quality in the finance sector.

* Investment Strategies: Major firms are making substantial investments in AI technologies, with companies like Meta Platforms, Inc. (META) focusing on AI applications to boost their future growth prospects. Investors are closely watching these developments, as the potential of AI to enhance traditional business models may drive stock prices significantly higher.

* Attracting Institutional Investments: Stocks like Palantir Technologies Inc. (PLTR) are gaining attention from billionaire hedge fund managers and analysts who foresee considerable upside potential due to the demand for AI solutions. This interest suggests that AI capabilities are not just enhancing current offerings but are also seen as critical components for future success in the marketplace.

* Market Performance: Palantir has experienced substantial stock price increases, attributed to its ability to integrate AI capabilities into its offerings. The growth reflects investor confidence in companies that successfully implement AI, even amidst broader market volatility.

* Content Production and Engagement: In the entertainment sector, companies like Netflix, Inc. (NFLX) are utilizing generative AI to simplify content production processes. As content creation becomes increasingly complex and competitive, AI is viewed as a tool that can improve efficiency and drive higher engagement with audiences.

Overall, AI is playing a transformative role in how companies operate, attract investments, and engage with consumers, positioning itself as a critical element in both financial performance and strategic planning across various industries.


**Non-metadata answer:**

Artificial intelligence (AI) is playing a significant role in recent finance-related news stories by enabling companies to enhance their operations, optimize investment strategies, and improve decision-making processes. Here are several key aspects highlighted in the stories:

* Increased Productivity: AI technologies are recognized for their ability to boost productivity and reduce human error. Companies like Jack Henry are integrating AI-driven lending technologies to streamline processes in the financial sector.

* Investment Opportunities: Analysts and investors are closely monitoring AI investments made by technology giants like Meta Platforms. These investments are expected to unlock new revenue streams, potentially leading to significant stock appreciation as the market begins to value these AI capabilities.

* Billionaire Investments: The involvement of billionaire hedge fund managers in companies like Palantir and Upstart showcases the perceived potential of AI stocks. Analysts forecast that these companies could see substantial upside due to their innovative AI applications, such as Palantir's work with government and commercial clients and Upstart's AI-driven credit risk assessment.

* Comparison of AI Stocks: The financial performance of AI-focused companies, like Palantir and Snowflake, is being scrutinized. Despite market volatility, both companies have shown strong gains, indicating investor confidence in their AI capabilities.

* Content Production: AI's potential to streamline content production is highlighted through companies like Netflix. The integration of AI in their content strategy suggests a transformative impact on how content is generated and delivered, which could lead to increased viewership and revenue.

In summary, AI is emerging as a pivotal force in reshaping investment strategies, enhancing operational efficiency, and influencing the financial performance of tech companies, which collectively drives interest in AI-related stocks among investors.

**comparison between answers**

The metadata-enriched answer feels more professional and data-driven. Provides richer context and it is easier to trace back to original news articles

The non-metadata answer feels more like a high-level overview. Still informative but lacks precise sourcing, covers similar themes but in a more generalized way


In [8]:
q_three_df = pd.DataFrame({
    'Criteria': ['Clarity', 'Detail & Depth', 'Use of Context', 'Accuracy & Grounding', 'Relevance', 'Narrative flow'],
    'Without Metadata': ["General references make it harder to trace", "Covers similar themes but lacks supporting data",
                         "Relies on summarized content without structured context","Less verifiable statements" ,
                         "Relevant but slightly broader" , "Informative but more segmented",],
    'With Metadata': ['Clear attribution of claims to specific companies', 'Rich in specifics',
                      'Integrates contextual metadata to support and validate each example', "Strong",
                      "Highly relevant to finance-related AI trends", "Smooth and cohesive"]
})

display(Markdown(dataframe_to_markdown(q_three_df)))

| Criteria | Without Metadata | With Metadata |
| --- | --- | --- |
| Clarity | General references make it harder to trace | Clear attribution of claims to specific companies |
| Detail & Depth | Covers similar themes but lacks supporting data | Rich in specifics |
| Use of Context | Relies on summarized content without structured context | Integrates contextual metadata to support and validate each example |
| Accuracy & Grounding | Less verifiable statements | Strong |
| Relevance | Relevant but slightly broader | Highly relevant to finance-related AI trends |
| Narrative flow | Informative but more segmented | Smooth and cohesive |

### Q4: How is Microsoft being portrayed in news stories about artificial intelligence?

**Metadata answer:**
The provided context does not include any news stories or references about Microsoft and its portrayal in relation to artificial intelligence. Therefore, I cannot provide an answer regarding how Microsoft is being portrayed in those discussions. If you have specific news articles or sources related to Microsoft and AI, feel free to share, and I'll be glad to help analyze them

**Non-metadata answer:**
The provided context does not specifically mention Microsoft or its portrayal in news stories about artificial intelligence. Therefore, I cannot provide a direct analysis regarding how Microsoft is being portrayed in such articles. However, if you have additional context or specific articles about Microsoft and its AI initiatives, I would be happy to help analyze that information.

**comparison between answers**

Both results give a generic response saying there's no Microsoft-related info in the provided context, so neither of them is useful.


In [7]:
q_four_df = pd.DataFrame({
    'Criteria': ['Clarity', 'Detail & Depth', 'Use of Context', 'Accuracy & Grounding', 'Relevance', 'Narrative flow'],
    'Without Metadata': ["No Microsoft info", "Lack of Microsoft info", "No Microsoft info","No Microsoft info" ,"Irrelevant" , "No Microsoft info",],
    'With Metadata': ['No Microsoft info', 'Lack of Microsoft info', 'No Microsoft info', "No Microsoft info", "Irrelevant", "No Microsoft info"]
})

display(Markdown(dataframe_to_markdown(q_four_df)))

| Criteria | Without Metadata | With Metadata |
| --- | --- | --- |
| Clarity | No Microsoft info | No Microsoft info |
| Detail & Depth | Lack of Microsoft info | Lack of Microsoft info |
| Use of Context | No Microsoft info | No Microsoft info |
| Accuracy & Grounding | No Microsoft info | No Microsoft info |
| Relevance | Irrelevant | Irrelevant |
| Narrative flow | No Microsoft info | No Microsoft info |

### Q5: What financial news headlines connect Amazon with automation or logistics?

**Metadata answer:**
The financial news headlines that connect Amazon with automation or logistics include:

* Amazon's AI Roadmap With AWS CEO Garman: This article discusses how Amazon is leveraging artificial intelligence across all its operations, which includes aspects of logistics and automation within its extensive e-commerce and cloud services.

* Winners And Losers Of Q1: C.H. Robinson Worldwide (NASDAQ:CHRW) Vs The Rest Of The Air Freight and Logistics Stocks: While this headline focuses on C.H. Robinson, it fits into the logistics theme as it gives insight into the performance of logistics companies, which can be influenced by Amazon's operations and the broader impact of automation in the logistics sector.

These headlines highlight Amazon's commitment to integrating automation and AI into its logistics and operational frameworks.

**Non-metadata answer:**
The financial news headlines that connect Amazon with automation or logistics include:

* Amazon's AI Roadmap With AWS CEO Garman: This article discusses how every aspect of Amazon is leveraging artificial intelligence, which often includes automation technologies that can enhance logistics and operational efficiencies.

* Winners And Losers Of Q1: C.H. Robinson Worldwide (NASDAQ:CHRW) Vs The Rest Of The Air Freight and Logistics Stocks: While this headline primarily focuses on logistics, Amazon's involvement in the logistics space, particularly with its delivery and supply chain operations, can be inferred through comparisons with other logistics companies.

These headlines highlight Amazon's emphasis on AI, which is closely tied to automation and improving logistics efficiency.

**comparison between answers**

Both answers are exactly the same, the amount of information is short, but that's what was provided by the semactic search, the second point is not clearly connected to Amazon.  


In [10]:
q_five_df = pd.DataFrame({
    'Criteria': ['Clarity', 'Detail & Depth', 'Use of Context', 'Accuracy & Grounding', 'Relevance', 'Narrative flow'],
    'Without Metadata': ["Straightforward and easy to read", "General observations without deep connections",
                         "Relies on surface-level context", "More interpretive and less anchored in verifiable metadata" ,
                         "Not so relevant" , "Slightly segmented but still coherent"],
    'With Metadata': ["Straightforward and easy to read", "General observations without deep connections",
                         "Relies on surface-level context", "More interpretive and less anchored in verifiable metadata" ,
                         "Not so relevant" , "Slightly segmented but still coherent"],
})

display(Markdown(dataframe_to_markdown(q_five_df)))

| Criteria | Without Metadata | With Metadata |
| --- | --- | --- |
| Clarity | Straightforward and easy to read | Straightforward and easy to read |
| Detail & Depth | General observations without deep connections | General observations without deep connections |
| Use of Context | Relies on surface-level context | Relies on surface-level context |
| Accuracy & Grounding | More interpretive and less anchored in verifiable metadata | More interpretive and less anchored in verifiable metadata |
| Relevance | Not so relevant | Not so relevant |
| Narrative flow | Slightly segmented but still coherent | Slightly segmented but still coherent |

### Question 6: What are the main themes emerging in financial news about the semiconductor industry?

**Metadata answer:**
The main themes emerging in financial news about the semiconductor industry, particularly regarding ON Semiconductor Corporation, include:

* International Revenue Trends: There is a strong focus on how international revenue figures impact overall forecasts for companies like ON Semiconductor. Investors are keen to understand global market dynamics and how they influence profitability and growth prospects.

* Investor Attention and Market Sentiment: ON Semiconductor is attracting significant investor interest, indicating a favorable market sentiment. The increased attention from investors may relate to potential growth opportunities and the company's standing in the semiconductor space.

* Earnings Performance: Reports hint at concerns surrounding soft earnings, yet there appears to be resilience among shareholders. This suggests that investors may be looking beyond short-term earnings challenges, focusing instead on long-term potential and future performance.

* Analyst and Expert Evaluations: The evaluation of ON Semiconductor against broader market trends and alongside the stock picks of well-known investors (like Glenn Russell Dubin) reveals a competitive landscape where firms are assessed for upside potential in uncertain markets.

* Comparative Analysis Within the Sector: There is a comparative analysis with other semiconductor companies, highlighting relative performance and opportunities amid changing market environments. This includes looking at how ON Semiconductor stacks up against peers like Analog Devices and Vishay Intertechnology in terms of growth and revenue handling.

Overall, the semiconductor sector is characterized by discussions around revenue growth strategies, market positioning, earnings outlooks, and investor sentiment shaped by broader economic conditions.


**Non-metadata answer:**

The main themes emerging in financial news about the semiconductor industry, particularly regarding companies like ON Semiconductor Corp. (ON), include:

* International Revenue Trends: There is a strong focus on the analysis of international revenue trends for semiconductor companies. Investors are particularly interested in how these trends influence market forecasts and the potential impact on stock performance.

* Earnings Performance and Investor Sentiment: Recent articles highlight that despite some companies posting soft earnings, there seems to be a lack of concern among shareholders. This indicates a level of optimism or confidence in future performance.

* Increased Investor Attention: There has been a noticeable increase in interest from investors in certain semiconductor stocks, such as ON Semiconductor. This could be driven by market conditions or specific corporate developments that are attracting attention.

* Stock Picks from Influential Investors: The investment strategies of prominent investors, like Billionaire Glenn Russell Dubin, are shaping discussions within the industry. Mentioning stocks with significant upside potential suggests that investors are looking for high-growth opportunities in the semiconductor sector.

* Market Environment Evaluation: The current earnings season is seen as a strategic point for identifying companies that are effectively navigating the changing business environment. This implies a broader evaluation of performance among semiconductor firms and an indication of potential winners in the market.

Overall, these themes reflect a combination of market analysis, investor sentiment, and individual stock performance within the semiconductor industry.

**comparison between answers**

The metadata-enriched is a little bit more interesting to read, but because of the limited context I dont think it satisfies the scope of the question


In the non-metadata answer is very similar in quality to the metadata-enriched asnwer, limited by the provided context, with a bit less interesting narrative.

In [14]:
q_six_df = pd.DataFrame({
    'Criteria': ['Clarity', 'Detail & Depth', 'Use of Context', 'Accuracy & Grounding', 'Relevance', 'Narrative flow'],
    'Without Metadata': ['Clear but more generic', 'Limited', 'Makes good use of limited context',
                         "Strong grounding, easy to verify", "Relevant but limited",
                         "Feels more like a summary than a cohesive narrative"],
    'With Metadata': ["Clear and well-structured", "Limited",
                         "Makes good use of limited context","Strong grounding, easy to verify" ,
                         "Relevant but limited" , "Feels more like a summary than a cohesive narrative"]
})

display(Markdown(dataframe_to_markdown(q_six_df)))

| Criteria | Without Metadata | With Metadata |
| --- | --- | --- |
| Clarity | Clear but more generic | Clear and well-structured |
| Detail & Depth | Limited | Limited |
| Use of Context | Makes good use of limited context | Makes good use of limited context |
| Accuracy & Grounding | Strong grounding, easy to verify | Strong grounding, easy to verify |
| Relevance | Relevant but limited | Relevant but limited |
| Narrative flow | Feels more like a summary than a cohesive narrative | Feels more like a summary than a cohesive narrative |

### Question 7: What trends are being reported in the retail industry?

**Metadata answer:**
The retail industry is currently facing several concerning trends. First, there is a notable adaptation of business models by retailers as technology changes consumer shopping behaviors. This shift indicates an ongoing transformation in how products are marketed and sold.

Second, despite these adaptations, demand within the industry remains volatile, making retailers vulnerable to fluctuations in consumer spending. This uncertainty has been reflected in the overall performance of retail stocks, which have dropped by 13.7% over the past six months, significantly underperforming compared to the S&P 500’s decline of 5.5%.

Additionally, retailers are grappling with external factors such as trade tariffs, prompting many to alter their supply chains and implement price increases, which may affect consumer purchasing habits further. Overall, the trends highlight a challenging environment for retailers as they navigate economic cycles and consumer behavior changes.

**Non-metadata answer:**

The retail industry is currently experiencing several concerning trends:

* Volatility in Consumer Demand: There is significant uncertainty in consumer spending, leading to fluctuations in demand for retail products. This has caused retail stocks to decline by 13.7% over the past six months, which is notably worse than the S&P 500's decline of 5.5%.

* Adaptation to Technological Changes: Retailers are adapting their business models in response to changing shopping behaviors driven by technology. This adaptation is essential as consumer preferences evolve, but it also contributes to the uncertainty faced by the sector.

* Impact of Trade Policies: Retail executives are dealing with the ramifications of tariffs and trade policies. They have reportedly shifted their supply chains, and many price increases have already been implemented in response to these external pressures.

* Economic Cycle Dependency: The performance of consumer discretionary businesses is closely linked to economic cycles. Recently, these businesses have faced headwinds, with an industry-wide drop of 12.3%, significantly worse than the S&P 500's 2.1% decline.

Overall, the combination of increased operational costs due to tariffs, changing consumer behaviors, and economic uncertainties has created a challenging environment for retailers.

**comparison between answers**
* The metadata-enriched answer performs better in terms of coherence, insight, and strategic framing.
* The non-metadata answer offers more granular data but lacks the interpretive layer that makes the trends feel interconnected and meaningful.


In [ ]:
q_seven_df = pd.DataFrame({
    'Criteria': ['Clarity', 'Datail & Depth', 'Use of Context', 'Accuracy & Grounding', 'Relevance', 'Narrative flow'],
    'Without Metadata': ["Clear and readable, though slightly more segmented than the metadata version", "Includes solid detail, but there some data not present in the context",
                         "Context is present but less integrated","the source of figures is less transparent" ,"All content is relevant" ,
                         "Slightly choppier flow due to bullet-style formatting and less connective phrasing.",],
    'With Metadata': ['The response is clear and well-structured, with smooth transitions between ideas.', 'Offers strong detail',
                      'Effectively integrates context', "Good grounding in factual sources", "All points directly address the question", "The answer flows logically"]
})

display(Markdown(dataframe_to_markdown(q_seven_df)))

| Criteria | Without Metadata | With Metadata |
| --- | --- | --- |
| Clarity | Clear and readable, though slightly more segmented than the metadata version | The response is clear and well-structured, with smooth transitions between ideas. |
| Datail & Depth | Includes solid detail, but there some data not present in the context | Offers strong detail |
| Use of Context | Context is present but less integrated | Effectively integrates context |
| Accuracy & Grounding | the source of figures is less transparent | Good grounding in factual sources |
| Relevance | All content is relevant | All points directly address the question |
| Narrative flow | Slightly choppier flow due to bullet-style formatting and less connective phrasing. | The answer flows logically |

### Q8: What risks or challenges are discussed in recent news about the energy industry?

**Metadata answer**

Recent news highlights several risks and challenges facing the energy industry, particularly regarding renewable energy and oilfield services:

* Legislative Threats to Renewable Energy Subsidies: The advancement of a bill by the U.S. House of Representatives aims to repeal significant subsidies for the renewable energy sector. If enacted, this legislation could render many renewable projects uneconomical, jeopardizing the financial viability of companies like NextEra Energy, Inc. (NEE) and Enphase Energy, Inc. (ENPH).

* Declining Oil Prices and Budget Cuts: The oilfield services sector, including companies like Halliburton Company (HAL) and Baker Hughes Company (BKR), faces a difficult environment due to sliding oil prices. This decline is compounded by increasing tariffs and shrinking drilling budgets, raising concerns about future profitability and operational sustainability.

* Uncertainty from Geopolitical and Trade Conflicts: The ongoing trade conflict, particularly related to tariffs, adds uncertainty to the market dynamics for companies reliant on critical materials like copper. For instance, Freeport-McMoRan Inc. (FCX) navigates the complexities of tariff implications on the industrial economy, which could affect competitive positioning and operational costs.

Overall, the energy industry is grappling with potential regulatory changes, volatile market conditions, and the impact of global trade policies.

**Non-metadata anwser:**

Recent news about the energy industry highlights several key risks and challenges, particularly for the renewable energy sector and oilfield services.

* Legislative Changes: The U.S. Congress is advancing a bill that could repeal important subsidies for renewable energy. This legislative change poses a significant risk to the economic viability of renewable energy projects, potentially making them uneconomical and prompting a crash in renewable energy stocks.

* Price Fluctuations: The oilfield service sector, represented by companies like SLB, HAL, and BKR, faces challenges due to sliding oil prices. This decline in prices can lead to reduced profitability and budget cuts in drilling operations.

* Increase in Tariffs: Rising tariffs are complicating the operational landscape for energy companies. As tariffs increase, costs related to imports and materials may rise, affecting profit margins and global competitiveness.

* Budget Constraints: With reduced drilling budgets, companies in the oilfield service sector may have to navigate a more challenging financial environment, limiting their operational capacity and future investments.

* Market Demand Shifts: Companies are also grappling with fluctuating demand for traditional energy sources versus renewable energy. As the market evolves, businesses must adapt or risk being left behind.

These combined factors create a highly uncertain environment for stakeholders in both the renewable energy and oilfield service sectors.


**comparison between answers**

Both responses are strong and cover a range of topics. The answer without metadata is more granular, while the one with metadata presents more cohesive and analytical information, offering a fluid narrative with well-connected insights. Both provide details about the companies relevant to each theme, but the metadata-enriched response does so more effectively, more frequently, and with greater precision.
- Clear presentation, slightly more engaging due to the analytical depth
- Well-detailed, covering a broad variety of themes
- Good use of context, with effective integration of metadata
- More challenging to link back to sources and related companies
- Highly relevant overall


In [15]:
q_eight_df = pd.DataFrame({
    'Criteria': ['Clarity', 'Detail & Depth', 'Use of Context', 'Accuracy & Grounding', 'Relevance', 'Narrative flow'],
    'Without Metadata': ["Clear presentation", "Covers a broad variety of themes", "Good use of context",
                         "More challenging to link back to sources and related companies" ,"Highly relevant overall" , "Provides granularity, a bit less interesting to read",],
    'With Metadata': ['Clear presentation, slightly more engaging due to the analytical depth',
                      'Covers a broad variety of themes', 'Good use of context, with effective integration of metadata',
                      "Easier to link back to sources and related companies", "Highly relevant overall", "Provides more insight"]
})

display(Markdown(dataframe_to_markdown(q_eight_df)))

| Criteria | Without Metadata | With Metadata |
| --- | --- | --- |
| Clarity | Clear presentation | Clear presentation, slightly more engaging due to the analytical depth |
| Detail & Depth | Covers a broad variety of themes | Covers a broad variety of themes |
| Use of Context | Good use of context | Good use of context, with effective integration of metadata |
| Accuracy & Grounding | More challenging to link back to sources and related companies | Easier to link back to sources and related companies |
| Relevance | Highly relevant overall | Highly relevant overall |
| Narrative flow | Provides granularity, a bit less interesting to read | Provides more insight |